In [1]:
from project_utils import  *
import pandas as pd

with open("data/train_data_embedded.json", "r") as f:
    train_data = pd.read_json(f, lines=True)


/home/jerryjose/miniconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
rel_features = ['metric_name', 'user_prompt', 'response', 'system_prompt']
#create all possible 2 combinations of rel_features
ed_rel_features = []
for i in range(len(rel_features)):
    for j in range(i + 1, len(rel_features)):
        ed_rel_features.append((rel_features[i], rel_features[j]))
ed_only_config = {'euclidean_distances': ed_rel_features,}


fe = FeatureEngineer(train_data, ed_only_config)
df_features = fe.create_features()
df_features.set_index('index')

data_gen = DataGenerator(df_features, test_fraction=0.2, fold_k=4, metric_column='main_metric')
dataset = data_gen.generate_splits()
cross_val_set = dataset['cv_splits']
cross_val_set[0][0]['score']

#train a simple lightgbm model on the cross validation set
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

model = LGBMRegressor()

X_train,y_train  = prepare_final(cross_val_set[0][0])
X_test, y_test = prepare_final(cross_val_set[0][1])

model.fit(X_train,y_train)

preds_base_model = model.predict(X_test).round(0)
mse_base_model = mean_squared_error(y_test, preds_base_model)

print(f"MSE:{mse_base_model}")


#display first 5 entries of preds and y_test side by side

